<a href="https://colab.research.google.com/github/juancuevas-ops/analisis-molecular-covid19/blob/main/Conexion__api_blast_Alinemiento_Proteina_S_Vacuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# alineamiento de esta proteina S  ruta  /content/protein_S.faa en blast conectandome  a la api
# url de los datos de Proteina S
!pip install Bio
from Bio import SeqIO
from Bio.Blast import NCBIXML
import time
import random
import requests
from io import StringIO

def buscar_dominios_union_calcio(archivo_fasta, api_key):
    """
    Busca dominios de unión a calcio en secuencias de proteínas de un archivo FASTA usando la API de NCBI.
    """
    for record in SeqIO.parse(archivo_fasta, "fasta"):
        secuencia = str(record.seq)
        print(f"\nAnalizando secuencia: {record.id}")

        max_sequence_length = 2000
        secuencias_partes = [secuencia[i:i + max_sequence_length] for i in range(0, len(secuencia), max_sequence_length)]

        resultados_totales = []

        for parte_secuencia in secuencias_partes:
            attempts = 0
            max_attempts = 5
            base_delay = 5
            while attempts < max_attempts:
                try:
                    url = "https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi"
                    params = {
                        "CMD": "Put",
                        "DATABASE": "nr", # Se cambió la base de datos a "nr"
                        "PROGRAM": "blastp",
                        "SEQUENCE": parte_secuencia,
                        "API_KEY": api_key,
                        "OUTPUT_FORMAT": "11"
                    }
                    response = requests.post(url, data=params)
                    response.raise_for_status()

                    if "<RID>" in response.text and "</RID>" in response.text:
                        rid = response.text.split("<RID>")[1].split("</RID>")[0]
                        print(f"RID obtenido: {rid}")
                        time.sleep(20)

                        url_get = "https://blast.ncbi.nlm.nih.gov/blast/Blast.cgi"
                        params_get = {
                            "CMD": "Get",
                            "RID": rid,
                            "API_KEY": api_key,
                            "FORMAT_TYPE": "XML"
                        }
                        response_get = requests.get(url_get, params=params_get)
                        response_get.raise_for_status()

                        resultados_blast_xml = NCBIXML.parse(StringIO(response_get.text))
                        resultados_totales.extend(list(resultados_blast_xml))
                        break
                    else:
                        print("RID no encontrado en la respuesta. Respuesta del servidor:")
                        print(response.text)
                        attempts += 1
                        delay = base_delay * (2 ** (attempts - 1)) + random.random()
                        time.sleep(delay)

                except requests.exceptions.RequestException as e:
                    attempts += 1
                    delay = base_delay * (2 ** (attempts - 1)) + random.random()
                    print(f"Error de conexión a NCBI: {e}. Reintentando en {delay:.2f} segundos (intento {attempts}/{max_attempts})...")
                    time.sleep(delay)
            else:
                print("Fallo al conectar con NCBI después de varios intentos para una parte de la secuencia.")
                continue

        for resultado in resultados_totales:
            if resultado.alignments:
                for alineamiento in resultado.alignments:
                    for hsp in alineamiento.hsps:
                        if "EF-hand" in alineamiento.title or "C2 domain" in alineamiento.title or "Gla domain" in alineamiento.title or "C-lectin domain" in alineamiento.title:
                            print(f"  Dominio de unión a calcio encontrado: {alineamiento.title}")
                            print(f"    Inicio: {hsp.query_start}, Fin: {hsp.query_end}")
                            print(f"    Alineamiento: {hsp.query[0:70]}") # Muestra los primeros 70 caracteres del alineamiento


# arcivo y Api
archivo_fasta = "/content/protein_S.faa"
api_key = "41e5fab4a67fa04a9b2dd3110841beda5807" # Reemplaza con tu API Key
buscar_dominios_union_calcio(archivo_fasta, api_key)



Analizando secuencia: YP_009724390.1
RID no encontrado en la respuesta. Respuesta del servidor:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta name="jig" content="ncbitoggler ncbiautocomplete"/>
<meta name="ncbi_app" content="static" />
<meta name="ncbi_pdid" content="blastformatreq" />
<meta name="ncbi_stat" content="false" />
<meta name="ncbi_sessionid" content="219616E87F1157A1_0000SID" />
<meta name="ncbi_phid" content="219616E87F1157A10000000000000001" />
<title>NCBI Blast</title>
<meta http-equiv="Pragma" content="no-cache">
<link rel="stylesheet" type="text/css" href="css/uswds.min.css" media="screen" />
<link rel="stylesheet"  type="text/css" href="https://www.ncbi.nlm.nih.gov/style-guide/static/nwds/css/nwds.css"/>

<link rel="stylesheet" href="css/heade

# Datos de Proteina S

https://www.ncbi.nlm.nih.gov/gene/43740568?log$=activity


Gene symbol
S
Gene description
surface glycoprotein
Locus tag
GU280_gp02
Gene type
protein coding
RefSeq status
PROVISIONAL
Organism
Severe acute respiratory syndrome coronavirus 2 (isolate: Wuhan-Hu-1, nat-host: Homo sapiens)
Lineage
Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Sarbecovirus
Also known as
spike glycoprotein
Summary
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is an enveloped, positive-sense, single-stranded RNA virus that causes coronavirus disease 2019 (COVID-19). Virus particles include the RNA genetic material and structural proteins needed for invasion of host cells. Once inside the cell the infecting RNA is used to encode structural proteins that make up virus particles, nonstructural proteins that direct virus assembly, transcription, replication and host control and accessory proteins whose function has not been determined.~ The structural proteins of SARS-CoV-2 include the envelope protein (E), spike or surface glycoprotein (S), membrane protein (M) and the nucleocapsid protein (N). The spike glycoprotein is found on the outside of the virus particle and gives coronavirus viruses their crown-like appearance. This glycoprotein mediates attachment of the virus particle and entry into the host cell. S protein is an important target for vaccine development, antibody therapies and diagnostic antigen-based tests.
NEW
Try the new Gene table
Try the new Transcript table
Genomic context
Sequence: NC_045512.2 (21563..25384)
NC_045512.2Genomic Context describing neighboring genes


# Resumen gráfico DE Conexion  la API de BLAST

Se muestra una descripción general de las secuencias de base de datos alineadas con la secuencia de consulta. La puntuación de cada alineación es indicado por uno de los cinco colores diferentes, que divide el rango de puntuaciones en cinco grupos. Múltiple Los segmentos de alineaciones a la misma secuencia de base de datos están conectados por una línea gris delgada. Pasar el ratón por encima de una secuencia de aciertos provoca La definición y la puntuación se mostrarán en la ventana de la parte superior, al hacer clic en una secuencia de aciertos se lleva al usuario a las alineaciones asociadas.

Función CDS
Al marcar esta opción, se permitirá que el formateador BLAST analice las entidades de secuencia anotadas que se encuentran en las proximidades o alrededor de ellas de aciertos y mostrarlos dentro del resultado de BLAST. En el caso de las secuencias de consulta personalizadas, también traducirá el CDS mediante el CDS traducción anotada en la secuencia de base de datos coincidente como guía. La falta de coincidencia en la traducción se resaltará en rosa. Un A continuación se da un ejemplo representativo con la traducción CDS.

gi|46452254|gb|AY585334.1| Sus scrofa cystic fibrosis transmembrane conductance regulator
(CFTR) mRNA, complete cds
Length=4449

Score = 5453 bits (2751),  Expect = 0.0
Identities = 4036/4449 (90%), Gaps = 6/4449 (0%)
Strand=Plus/Plus

CDS: Putative 1       1      M  Q  R  S  P  L  E  K  A  S  V  V  S  K  L  F  F  S  W  T
Query                 133   ATGCAGAGGTCGCCTCTGGAAAAGGCCAGCGTTGTCTCCAAACnnnnnnnCAGCTGGACC  192
                            |||||||||||||||||||||||||||||| |  ||||||||||||||||||||||||||
Sbjct                 1     ATGCAGAGGTCGCCTCTGGAAAAGGCCAGCATCTTCTCCAAACTTTTTTTCAGCTGGACC  60
CDS:cystic fibrosis   1      M  Q  R  S  P  L  E  K  A  S  I  F  S  K  L  F  F  S  W  T

CDS: Putative 1       21     R  P  I  L  R  K  G  Y  R  Q  R  L  E  L  S  D  I  Y  Q  I
Query                 193   AGACCAATTTTGAGGAAAGGATACAGACAGCGCCTGGAATTGTCAGACATATACCAAATC  252
                            |||||||||||||| |||||||| |||||||||||||||||||||||||||||||| |||
Sbjct                 61    AGACCAATTTTGAGAAAAGGATATAGACAGCGCCTGGAATTGTCAGACATATACCATATC  120
CDS:cystic fibrosis   21     R  P  I  L  R  K  G  Y  R  Q  R  L  E  L  S  D  I  Y  H  I

CDS: Putative 1       41     P  S  V  D  S  A  D  N  L  S  E  K  L  E  R  E  W  D  R  E
Query                 253   CCTTCTGTTGATTCTGCTGACAATCTATCTGAAAAATTGGAAAGAGAATGGGATAGAGAG  312
                            |||||  ||| |||||||||||||| |||||||||||||||||||||||||| |||||
Sbjct                 121   TCTTCTTCTGACTCTGCTGACAATCTGTCTGAAAAATTGGAAAGAGAATGGGACAGAGAA  180
CDS:cystic fibrosis   41     S  S  S  D  S  A  D  N  L  S  E  K  L  E  R  E  W  D  R  E
Enmascaramiento
Hay dos opciones que determinan la forma en que se debe mostrar la región enmascarada del filtro.

Carácter de enmascaramiento

X o N muestra la región enmascarada en X para la proteína y N para el nucleótido.

Las minúsculas muestran la región maksed en letras minúsculas.

Color de enmascaramiento

La región enmascarada se puede "resaltar" con fuentes de color gris o rojo.

Descripciones
Esta opción restringe el número de descripciones breves de secuencias coincidentes notificadas al número especificado. La configuración predeterminada varía de una página a otra. Véase también esperar.

# Vista de alineación Pairwise

Las alineaciones de las bases de datos se muestran como pares de coincidencias entre la consulta y la secuencia de asuntos. Una línea intermedia entre la consulta y la secuencia de asunto muestra el estado de una carta. Para la proteína alineaciones (por ejemplo, BLASTP/BLASTX/TBLASTN), las identidades presentan la letra, las sustituciones conservadoras presente un "+", y nada de lo contrario. Para alineaciones de nucleótidos (por ejemplo, BLASTN y megaBLAST) un "|" se muestra para las coincidencias y no para las discrepancias. Esta es la vista predeterminada.

# Emparejado con puntos para identidades

Las alineaciones de las bases de datos se anclan (se muestran en relación con) a la secuencia de consulta de forma emparejada con desajustes coloreados en rojo. Sbjct estará en rojo y negrita font si una línea de la alineación contiene discrepancias. Vea el ejemplo a continuación.

Anclado a la consulta con puntos para identidades

Las identidades se muestran como puntos (.), y las discrepancias se muestran como una sola letra Abreviaturas.

Anclado a consultas con letras para identidades

Las identidades se muestran como abreviaturas de nucleótidos de una sola letra.

Consulta plana anclada con puntos para identidades

La pantalla "plana" muestra las inserciones como eliminaciones en la consulta. Las identidades se muestran como puntos (.), con discrepancias que se muestran como abreviaturas de una sola letra.

Anclado a la consulta plana con letras para identidades

La pantalla "plana" muestra las inserciones como eliminaciones en la consulta. Las identidades se muestran como abreviaturas de una sola letra.

gi|21536448|ref|NM_002622.3| Homo sapiens prefoldin 1 (PFDN1), mRNA
Length=1296

Score =  392 bits (212),  Expect = 2e-107
Identities = 220/223 (98%), Gaps = 3/223 (1%)
Strand=Plus/Plus


Query  107  TCCTACCTGGAGCGAAG-GTTANAGGAAGCTGAGGACAACATCCGGGAGATGCTGATGGC  165
Sbjct  300  ................. C .... - .....................................  358

Query  166  ACGAAGGG-CCAGTAGGGAGCCTCTCTGGGAAGCTCTTCCTCCTGCCCCTCCCATTCCTG  224

Sbjct  359  ........ C ...................................................  418

Query  225  GTGGGGGCAGAGGAGTGTCTGCAGGGAAACAGCTTCTCCTCTGCCCCGATGGATGCTTTA  284
Sbjct  419  ............................................................  478

Query  285  TTTGGATGGCCTGGCAACATCACATTTTCTGCATCACCCTGAG  327
Sbjct  479  ...........................................  521
Descargar

Los enlaces de descarga permiten descargas de XML, informe de texto, CSV, XML, ASN.1 o JSON.

Formato para PSI-BLAST
El programa Position-Specific Iterated BLAST (PSI-BLAST) realiza búsquedas iterativas con una consulta de proteínas, en el que las secuencias encontradas en una ronda de búsqueda se utilizan para crear un modelo de puntuación personalizado para la siguiente ronda.

En PSI-BLAST, el algoritmo no está vinculado a una matriz de puntuación específica, como BLOSUM62, que se ha implementado utilizando una matriz de sustitución AxA donde A es el tamaño del alfabeto. En su lugar, utiliza una matriz QxA, donde Q es la longitud de la secuencia de consulta. En cada posición, el costo de una letra depende de la posición con respecto a la consulta y la letra en la secuencia de asuntos.

Para ejecutar esta búsqueda, se debe marcar la casilla de verificación "Formato para PSI-BLAST".

Umbral de inclusión
Esto establece el umbral de significación estadística para incluir una secuencia en el modelo utilizado por PSI-BLAST para crear el PSSM en la siguiente iteración.

Limitar resultados por consulta entrez
Esta función es similar a la "Limitar por términos de consulta de Entrez" en la sección de opciones. La única diferencia es que Solo se aplica a las visitas identificadas. En otras palabras, se aplica después de la búsqueda y permite a los usuarios ver solo hits que se ajustan al requisito de los términos de consulta de Entrez. El valor predeterminado es formatear sin entrada los términos de consulta y Permitir a los usuarios ver todas las visitas.

# Rango de valor esperado
Esto indica al formateador BLAST que muestre las visitas con el valor Esperado dentro del intervalo especificado. El valor predeterminado es 0 para la configuración del valor esperado. El límite inferior va a la primera casilla, el límite superior va a la segunda casilla

# Resumen: se encuentra similitud de un 98% de similitud Proteina S  con fibrosis Quistica:

La fibrosis quística es una enfermedad hereditaria que afecta principalmente a los pulmones y al sistema digestivo. Causa que el cuerpo produzca una mucosidad espesa y pegajosa que obstruye los conductos y causa una variedad de síntomas.

Aquí te presento los principales síntomas de la fibrosis quística, organizados por sistemas del cuerpo:

# Síntomas Respiratorios:

Tos persistente: Puede ser una tos seca o una tos que produce mucosidad espesa (esputo).
Sibilancias: Un sonido silbante al respirar.
Falta de aliento: Dificultad para respirar, especialmente durante el ejercicio.
Infecciones pulmonares recurrentes: Como bronquitis o neumonía.
Congestión nasal: Inflamación o congestión de las fosas nasales.
Sinusitis recurrente: Infecciones frecuentes de los senos paranasales.
Síntomas Digestivos:

Heces grasosas y con mal olor: Debido a la mala absorción de las grasas.
Poco aumento de peso y crecimiento: Dificultad para ganar peso o crecer normalmente.
Obstrucción intestinal en recién nacidos: Conocido como íleo meconial.
Estreñimiento crónico o severo: Dificultad para evacuar las heces.
Dolor abdominal: Debido a problemas intestinales.
Prolapso rectal: Cuando parte del recto sobresale por el ano.
Otros Síntomas:

Piel salada: El sudor de las personas con fibrosis quística es mucho más salado de lo normal.
Dedos en palillo de tambor: Ensanchamiento y redondeamiento de las puntas de los dedos de las manos y los pies.
Pancreatitis: Inflamación del páncreas.
Problemas de fertilidad: Especialmente en los hombres.
Retraso de la pubertad: En algunos casos.
Es importante recordar:

Los síntomas de la fibrosis quística pueden variar de una persona a otra en gravedad.
Algunos síntomas pueden aparecer en la infancia, mientras que otros pueden desarrollarse más tarde.
Si tú o alguien que conoces experimenta síntomas que podrían indicar fibrosis quística, es crucial buscar atención médica para un diagnóstico y tratamiento adecuados.




Algoritmo Creado Alejandor Cuevas Vasquez
https://www.kaggle.com/pollicio/code